## Using the EWXPWSDB Collector Class

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

- creating a temporary DB
- using the collector class for existing station records to
    - get past data 
    - pull data for short period, e.g. from a scheduler
    - get all recent data, aka catch up data from last record


In [ ]:
%load_ext autoreload
%autoreload 2

## Constants/Config

In [ ]:

station_file = '../data/test_stations.tsv'
station_type = 'ZENTRA'
station_code = 'EWX-METER-FO2'

In [ ]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse
from ewxpwsdb.db.importdata import import_station_file
from ewxpwsdb.collector import Collector
from ewxpwsdb.time_intervals import UTCInterval
from sqlmodel import select

## create engine temp sqlite database

In [ ]:
# this notebook does not use the .env file or the get_db_url() function for the app's db, 
# instead it creates a new file to be used for a temporary, empty sqlite database, deleted in the end. 
from tempfile import NamedTemporaryFile

temp_sqlitefile = NamedTemporaryFile(suffix = '.db')
temp_db_url = f"sqlite:///{temp_sqlitefile.name}"
engine = get_engine(db_url=temp_db_url)
print(engine)
init_db(engine)
import_station_file(station_file, engine = engine)


In [ ]:
def get_one_station(station_type, station_code = None, engine=engine):
    """ global engine is default"""

    if station_code:
        statement = select(WeatherStation).where(WeatherStation.station_code == station_code)
    else:
        statement = select(WeatherStation).where(WeatherStation.station_type == station_type)

    with Session(engine) as session:
        results = session.exec(statement)
        weather_station = results.first()

    return weather_station

In [ ]:
station = get_one_station(station_type, station_code, engine)
print(station.station_code)


In [ ]:
collector = Collector(station, engine)
collector.weather_api

In [ ]:
collector._session.commit()

In [ ]:
from datetime import timedelta
from ewxpwsdb.time_intervals import previous_fourteen_minute_interval

interval = previous_fourteen_minute_interval()

interval.start = interval.start - timedelta(hours = 1)
interval.end = interval.end - timedelta(hours = 0.5)
interval

In [ ]:
# call the method to see if it's working
somerex = collector.request_and_store_weather_data_utc(interval)
somerex

In [ ]:
collector.current_api_response_record_ids

In [ ]:
# there is no guarantee this is not going to be blank due to SQLAlchemy ORM cache rules, especially if the data has to be split in an array
collector.current_api_response

In [ ]:
collector.current_readings

### Clean up

remove test database

In [ ]:
if collector:
    collector._session.close()
    collector._engine.dispose()

engine.dispose()

temp_sqlitefile.close()
collector = None
